#  Converting SAS Data into SQLite and Use SQL Queries
================================================================================

In [4]:
import pandas as pd
import pyreadstat
import sqlite3 

In [31]:
%%time
df2, meta = pyreadstat.read_sas7bdat('../Data/modpop_1to1_201910.sas7bdat', row_limit= 50000, encoding='latin1')

CPU times: user 1.07 s, sys: 53.6 ms, total: 1.12 s
Wall time: 1.12 s


In [32]:
meta.number_rows

50000

In [33]:
meta.column_names

['ACTG_PER',
 'COMPLETION_DTE',
 'DARTS_CASE_ID',
 'DLQ_BF_CONVERSION',
 'DLQ_BF_TRIAL',
 'DLQ_RESET_DTE',
 'DLQ_VALID_DTE',
 'FNMA_LN',
 'INT_FORGIVEN',
 'LIQD_DTE',
 'MOD_CURR_DTE',
 'MOD_START_DTE',
 'MOD_TYPE',
 'PCT_PAY_CHG',
 'POST_MOD_PMT',
 'POST_MOD_PRDCT',
 'POST_MOD_RATE',
 'POST_MOD_REMTRM',
 'POST_MOD_UPB',
 'PRE_MOD_PMT',
 'PRE_MOD_PRDCT',
 'PRE_MOD_RATE',
 'PRE_MOD_REMTRM',
 'PRE_MOD_UPB',
 'TOT_MTHY_HHLD_GRS_INCM_AMT',
 'TRIAL_END_DTE',
 'TRIAL_OUTCOME',
 'TRL_FST_PMT_DTE',
 'UPB_FORBORN',
 'UPB_FORGIVEN',
 'STEP_RATE_01',
 'STEP_DATE_01',
 'STEP_RATE_02',
 'STEP_DATE_02',
 'STEP_RATE_03',
 'STEP_DATE_03',
 'STEP_RATE_04',
 'STEP_DATE_04',
 'STEP_RATE_05',
 'STEP_DATE_05',
 'STEP_RATE_06',
 'STEP_DATE_06',
 'STEP_RATE_07',
 'STEP_DATE_07',
 'STEP_RATE_08',
 'STEP_DATE_08',
 'STEP_RATE_09',
 'STEP_DATE_09',
 'STEP_RATE_10',
 'STEP_DATE_10',
 'DARTS_CPGN_ID']

https://ofajardo.github.io/pyreadstat_documentation/_build/html/index.html

In [34]:
df2.head()

,ACTG_PER,COMPLETION_DTE,DARTS_CASE_ID,DLQ_BF_CONVERSION,DLQ_BF_TRIAL,DLQ_RESET_DTE,DLQ_VALID_DTE,FNMA_LN,INT_FORGIVEN,LIQD_DTE,...,STEP_DATE_06,STEP_RATE_07,STEP_DATE_07,STEP_RATE_08,STEP_DATE_08,STEP_RATE_09,STEP_DATE_09,STEP_RATE_10,STEP_DATE_10,DARTS_CPGN_ID
0,201910.0,2001-02-01,5.020100e+09,5.0,NaN,2001-02-01,2001-02-01,1010059174,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,201910.0,2004-04-01,1.020402e+09,7.0,NaN,2004-04-01,2004-04-01,1010059435,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,201910.0,1999-12-01,5.029904e+09,9.0,NaN,2000-01-01,2000-02-01,1010068307,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,201910.0,1998-04-01,5.029801e+09,2.0,NaN,1998-04-01,1998-06-01,1010068943,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,201910.0,2005-02-01,4.020502e+09,19.0,NaN,2005-02-01,2005-02-01,1010072748,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [35]:
print("Number of rows: ", meta.number_rows, '\t', "Number of colums: ", meta.number_columns, '\t', "Encoding: ", meta.file_encoding )
#print(df2.dtypes)
for x,y in zip(meta.column_names, meta.column_labels):
    print(x,"\t\t\t ",y)

Number of rows:  50000 	 Number of colums:  51 	 Encoding:  latin1
ACTG_PER 			  Accounting Period
COMPLETION_DTE 			  Date When a Trial Completes
DARTS_CASE_ID 			  Unique ID Assigned to Each Workout
DLQ_BF_CONVERSION 			  DELQ of a Loan before MOD_CURR_DTE
DLQ_BF_TRIAL 			  DELQ of a Loan before Trial Period
DLQ_RESET_DTE 			  Date When SIR Resets the Delinquency
DLQ_VALID_DTE 			  Date Starting When DLQ Is Valid
FNMA_LN 			  Fannie Mae Loan ID
INT_FORGIVEN 			  Interest Forgiven as Part of Workout
LIQD_DTE 			  Loan Liquidation Date
MOD_CURR_DTE 			  Date When a Modified Loan Becomes Current
MOD_START_DTE 			  Date When a Trial MOD Starts
MOD_TYPE 			  Type of Modification
PCT_PAY_CHG 			  Pctg of Payment Change Before/After MOD
POST_MOD_PMT 			  Scheduled Monthly Payment After MOD
POST_MOD_PRDCT 			  Product Code After Workout
POST_MOD_RATE 			  Note Rate After Workout
POST_MOD_REMTRM 			  Scheduled Remaining Term After MOD
POST_MOD_UPB 			  Total UPB after MOD, including FB AMT
PR

In [36]:
%%time
conn = sqlite3.connect('modpop.sqlite3')

df2.to_sql('modpop', conn, if_exists='replace', index=False)


CPU times: user 3.68 s, sys: 74.3 ms, total: 3.75 s
Wall time: 3.86 s


In [37]:
df_sum = pd.read_sql("select MOD_Type, count(*) as total from modpop group by MOD_type", conn)


In [38]:
df_sum

,MOD_TYPE,total
0,ALT,133
1,ALT2011,57
2,ALT3.0,709
3,CAPEX,662
4,CAPEXD,37
5,EXTENDD,132
6,FLEXMOD,937
7,FLEXMODD,5
8,FNMAMOD,1380
9,HAMP,1195


In [21]:
pip install ipython-sql --user

Looking in indexes: https://nexusrepository.fanniemae.com/nexus/repository/pypi-all/simple
Note: you may need to restart the kernel to use updated packages.


In [39]:
%load_ext sql


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [40]:
%sql sqlite:///./modpop.sqlite3

'Connected: @./modpop.sqlite3'

In [41]:
%%sql
select MOD_Type, count(*) from modpop group by MOD_type;

 * sqlite:///./modpop.sqlite3
Done.


MOD_TYPE,count(*)
ALT,133
ALT2011,57
ALT3.0,709
CAPEX,662
CAPEXD,37
EXTENDD,132
FLEXMOD,937
FLEXMODD,5
FNMAMOD,1380
HAMP,1195


In [43]:
%%sql
-- drop table mod_summary;
create table mod_summary as 
select MOD_Type, count(*) from modpop group by MOD_type;

select * from mod_summary where mod_type like 'ALT%';

 * sqlite:///./modpop.sqlite3
(sqlite3.OperationalError) table mod_summary already exists
[SQL: -- drop table mod_summary;
create table mod_summary as 
select MOD_Type, count(*) from modpop group by MOD_type;]
(Background on this error at: http://sqlalche.me/e/e3q8)


## Lets have a quick look at command prompt
1. Login to SQLite
2. Run .help
3. Run .tables, .schema, .databases
4. Describe table with `pragma table_info('tablename');` . Set .header on and .mode column
5. Export data to CSV using .mode csv and .output myfile.csv